<!-- #### Step 1: remove rows where food_desc == ingredient_dec -->

### **Approach 1 (did not work): making API calls to `food.com` to find if a corresponding url can be retrieved for a food item**

Using the name of the food and try to look up on food.com directly kept returning error code 200

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

foods_df = pd.read_csv('../processed_data/foods_list.csv.csv')

# Function to search Food.com for a specific food item
def find_recipe_on_food_com(food_desc):
    # Format the search URL
    search_url = f"https://www.food.com/search/{food_desc.replace(' ', '%20')}"
    
    try:
        # Send a GET request to Food.com
        response = requests.get(search_url, headers={'User-Agent': 'Mozilla/5.0'})
        
        # Debugging: Print status code
        print(f"Requesting {search_url}, Status Code: {response.status_code}")

        if response.status_code == 200:
            # Parse the HTML response with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Look for recipe results
            recipe_cards = soup.find_all('div', class_='card__detailsContainer-left')
            if recipe_cards:
                # Extract the first recipe link as an example
                first_recipe = recipe_cards[0]
                recipe_link_tag = first_recipe.find('a', href=True)
                if recipe_link_tag:
                    recipe_url = "https://www.food.com" + recipe_link_tag['href']
                    return True, recipe_url
            return False, None
        else:
            print(f"Failed to retrieve results for {food_desc} (status code: {response.status_code})")
            return False, None
    except Exception as e:
        print(f"Error while searching for {food_desc}: {e}")
        return False, None

# Filter the foods dataframe and collect those that are found on Food.com
filtered_foods = []
matched_foods = []

for _, row in foods_df.iterrows():
    food_desc = row['food_desc']
    found, recipe_url = find_recipe_on_food_com(food_desc)
    if found:
        # Append row and the recipe URL to the list
        row_dict = row.to_dict()
        row_dict['recipe_url'] = recipe_url
        matched_foods.append(row_dict)
        print(f"Found recipe for '{food_desc}': {recipe_url}")
    else:
        filtered_foods.append(row)
    
    # Randomized sleep to avoid rate limiting
    time.sleep(random.uniform(3, 7))

filtered_foods_df = pd.DataFrame(filtered_foods)
matched_foods_df = pd.DataFrame(matched_foods)

filtered_foods_df.to_csv('../processed_data/food.com_filtered_foods_df.csv', index=False)
matched_foods_df.to_csv('../processed_data/food.com_matched_foods_df.csv', index=False)

Requesting https://www.food.com/search/Hard%20candy, Status Code: 200
Requesting https://www.food.com/search/Soft%20drink,%20root%20beer, Status Code: 200
Requesting https://www.food.com/search/Potato%20chips,%20baked,%20flavored, Status Code: 200
Requesting https://www.food.com/search/Ham,%20prepackaged%20or%20deli,%20luncheon%20meat,%20reduced%20sodium, Status Code: 200
Requesting https://www.food.com/search/Bread,%20wheat%20or%20cracked%20wheat, Status Code: 200
Requesting https://www.food.com/search/Mayonnaise-type%20salad%20dressing, Status Code: 200
Requesting https://www.food.com/search/Popcorn,%20air-popped,%20no%20butter%20added, Status Code: 200
Requesting https://www.food.com/search/Fruit%20flavored%20drink,%20powdered,%20reconstituted, Status Code: 200


KeyboardInterrupt: 

### **Approach 2 (worked but not quite satisfactorily): same as approach 1 but looking up on AllRecipes.com**

 also we should not use AllRecipes.com since it doesn't have the same nutrition facts as food.com

In [72]:
# !python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------------ --------------- 7.9/12.8 MB 37.3 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 34.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [75]:
import pandas as pd
import time
import re
from allrecipes import AllRecipes
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import spacy

# Load your dataset
file_path = '../processed_data/foods_list.csv'
foods_df = pd.read_csv(file_path)

# Load an open-source LLM (using GPT-Neo here as an example)
model_name = "EleutherAI/gpt-neo-1.3B"  # You can replace this with another open-source model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load spaCy model for fallback noun extraction
nlp = spacy.load("en_core_web_sm")

# Rule-based simplification function
def rule_based_simplify(food_desc):
    # Convert to lowercase for uniformity
    food_desc = food_desc.lower()

    # # Remove known unnecessary words and phrases
    # food_desc = re.sub(r'\b(no butter added|reduced sodium|low fat|unsweetened|packaged|prepackaged|deli|flavored|optional)\b', '', food_desc, flags=re.IGNORECASE)

    # Keep only the first 2-3 key terms (focus on the core description)
    simplified_name = " ".join(food_desc.split()[:3])

    # Remove conjunctions and unnecessary small words like "or", "and", "with", etc.
    simplified_name = re.sub(r'\b(or|and|with)\b', '', simplified_name)

    # Strip unnecessary commas, trailing or leading whitespaces
    simplified_name = re.sub(r',', '', simplified_name).strip()

    return simplified_name

# Function to extract the main food noun from the food description using a rule-based approach with spaCy, and fallback to LLM if necessary
def extract_main_noun(food_desc):
    # First try extracting with spaCy
    main_noun = extract_main_noun_spacy(food_desc)
    if main_noun:
        return main_noun

    # If spaCy fails, use the LLM
    prompt = f"From the following food description, extract only one word that is the main food noun: '{food_desc}'. Make sure it is the most important word that describes the food."
    try:
        response = generator(prompt, max_new_tokens=5, temperature=0.2, num_return_sequences=1)
        main_noun = response[0]['generated_text'].strip().split()[0]  # Extract the first word of the LLM response

        # Log the extracted main noun
        print(f"Extracted main noun: '{main_noun}' from '{food_desc}'")
        return main_noun

    except Exception as e:
        print(f"Error while extracting main noun for '{food_desc}' using LLM: {e}")
        return ""

# Function to extract the main noun using spaCy
def extract_main_noun_spacy(food_desc):
    doc = nlp(food_desc)
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    if nouns:
        main_noun = nouns[0]  # Choose the first noun as the main food item
        print(f"spaCy extracted main noun: '{main_noun}' from '{food_desc}'")
        return main_noun
    else:
        print(f"No noun found in '{food_desc}' using spaCy.")
        return ""

# Function to use allrecipes package to search AllRecipes.com for a specific food item
def find_recipe_on_allrecipes(food_desc):
    # Use combined approach to simplify the food description
    simplified_name = rule_based_simplify(food_desc)
    print(f"Simplified food name for AllRecipes: '{simplified_name}' from '{food_desc}'")

    try:
        # Use AllRecipes package to perform the search
        query_result = AllRecipes.search(simplified_name)

        # If no results found, print a message for debugging
        if not query_result:
            print(f"No recipe found for '{simplified_name}' on AllRecipes.")
            return False, None

        # Get the details of the first returned recipe
        main_recipe_url = query_result[0]['url']

        # Extract the main food noun for validation
        main_noun = extract_main_noun(food_desc)

        # Check if the main noun appears in the URL
        if main_noun and main_noun.lower() in main_recipe_url.lower():
            return True, main_recipe_url
        else:
            print(f"Recipe URL '{main_recipe_url}' does not match the main noun '{main_noun}' from '{food_desc}'.")
            return False, None

    except Exception as e:
        print(f"Error while searching for '{simplified_name}' on AllRecipes: {e}")
        return False, None

filtered_foods = []
matched_foods = []

for _, row in foods_df.iterrows():
    food_desc = row['food_desc']
    found, recipe_url = find_recipe_on_allrecipes(food_desc)

    if found:
        row_dict = row.to_dict()
        row_dict['recipe_url'] = recipe_url
        matched_foods.append(row_dict)
        print(f"Found recipe for '{food_desc}': {recipe_url}")
    else:
        filtered_foods.append(row)

    time.sleep(3)

filtered_foods_df = pd.DataFrame(filtered_foods)
matched_foods_df = pd.DataFrame(matched_foods)

filtered_foods_df.to_csv('../processed_data/filtered_foods_df.csv', index=False)
matched_foods_df.to_csv('../processed_data/matched_foods_df.csv', index=False)

print("Filtered data saved to 'filtered_foods_df.csv'")
print("Matched data saved to 'matched_foods_df.csv'")


Simplified food name for AllRecipes: 'hard candy' from 'Hard candy'
spaCy extracted main noun: 'candy' from 'Hard candy'
Found recipe for 'Hard candy': https://www.allrecipes.com/recipe/35842/hard-candy/
Simplified food name for AllRecipes: 'soft drink root' from 'Soft drink, root beer'
spaCy extracted main noun: 'drink' from 'Soft drink, root beer'
Recipe URL 'https://www.allrecipes.com/recipe/284771/old-fashioned-root-beer-slushy/' does not match the main noun 'drink' from 'Soft drink, root beer'.
Simplified food name for AllRecipes: 'potato chips baked' from 'Potato chips, baked, flavored'
spaCy extracted main noun: 'chips' from 'Potato chips, baked, flavored'
Recipe URL 'https://www.allrecipes.com/recipe/8848/potato-chip-chicken-casserole/' does not match the main noun 'chips' from 'Potato chips, baked, flavored'.
Simplified food name for AllRecipes: 'ham' from 'Ham, prepackaged or deli, luncheon meat, reduced sodium'
spaCy extracted main noun: 'Ham' from 'Ham, prepackaged or deli,

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


No noun found in 'Roll, white, soft' using spaCy.
Extracted main noun: 'From' from 'Roll, white, soft'
Recipe URL 'https://www.allrecipes.com/recipe/15295/soft-sugar-cookies-iv/' does not match the main noun 'From' from 'Roll, white, soft'.
Simplified food name for AllRecipes: 'fruit juice drink' from 'Fruit juice drink'
spaCy extracted main noun: 'Fruit' from 'Fruit juice drink'
Found recipe for 'Fruit juice drink': https://www.allrecipes.com/recipe/20699/exotic-fruit-drink/
Simplified food name for AllRecipes: 'egg salad made' from 'Egg Salad, made with any type of fat free dressing'
spaCy extracted main noun: 'type' from 'Egg Salad, made with any type of fat free dressing'
Recipe URL 'https://www.allrecipes.com/recipe/103767/jens-heavenly-egg-salad/' does not match the main noun 'type' from 'Egg Salad, made with any type of fat free dressing'.
Simplified food name for AllRecipes: 'grits instant made' from 'Grits, instant, made with water, no added fat'
spaCy extracted main noun: 'Gr

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


No noun found in 'Butter, tub' using spaCy.
Extracted main noun: 'From' from 'Butter, tub'
Recipe URL 'https://www.allrecipes.com/recipe/246358/johnsonville-brat-hot-tub/' does not match the main noun 'From' from 'Butter, tub'.
Simplified food name for AllRecipes: 'potato french fries' from 'Potato, french fries, fast food'
spaCy extracted main noun: 'fries' from 'Potato, french fries, fast food'
Found recipe for 'Potato, french fries, fast food': https://www.allrecipes.com/recipe/50223/homemade-crispy-seasoned-french-fries/
Simplified food name for AllRecipes: 'chili con carne' from 'Chili con carne, NS as to beans'
spaCy extracted main noun: 'carne' from 'Chili con carne, NS as to beans'
Found recipe for 'Chili con carne, NS as to beans': https://www.allrecipes.com/recipe/89993/award-winning-chili-con-carne/
Simplified food name for AllRecipes: 'soft drink fruit' from 'Soft drink, fruit flavored, caffeine free'
spaCy extracted main noun: 'drink' from 'Soft drink, fruit flavored, caff

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


No noun found in 'Mayonnaise, light' using spaCy.
Extracted main noun: 'From' from 'Mayonnaise, light'
Recipe URL 'https://www.allrecipes.com/recipe/24023/mayonnaise-biscuits/' does not match the main noun 'From' from 'Mayonnaise, light'.
Simplified food name for AllRecipes: 'queso asadero' from 'Queso Asadero'


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


No noun found in 'Queso Asadero' using spaCy.
Extracted main noun: 'From' from 'Queso Asadero'
Recipe URL 'https://www.allrecipes.com/recipe/237356/honey-lime-chicken-enchiladas/' does not match the main noun 'From' from 'Queso Asadero'.
Simplified food name for AllRecipes: 'grapefruit raw' from 'Grapefruit, raw'
No recipe found for 'grapefruit raw' on AllRecipes.
Simplified food name for AllRecipes: 'enchilada just cheese' from 'Enchilada, just cheese, meatless, no beans, green-chile or enchilada sauce'
spaCy extracted main noun: 'cheese' from 'Enchilada, just cheese, meatless, no beans, green-chile or enchilada sauce'
Recipe URL 'https://www.allrecipes.com/recipe/74188/green-chicken-enchilada/' does not match the main noun 'cheese' from 'Enchilada, just cheese, meatless, no beans, green-chile or enchilada sauce'.
Simplified food name for AllRecipes: 'enchilada sauce green' from 'Enchilada sauce, green'
spaCy extracted main noun: 'sauce' from 'Enchilada sauce, green'
Found recipe for 

KeyboardInterrupt: 

### **Approach 3: using Autogen to automate the process of judging whether a food is recommendable or not**

Helped reduce from 9.6k foods down to 7.5k foods and can be reduced further with manual filters from human reviewers

In [87]:
import os
import pandas as pd
import time
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": api_key
}

# Define the food recommendation agent
agent = ConversableAgent(
    name="food_recommendation_agent",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Define the criteria for what makes a food item recommendable
# We as the prompters can always add more criteria as needed
recommendable_criteria = """
The food description must represent a complete food item or dish that is suitable for recommendation to users. Avoid descriptions that are:
1. Too generic (e.g., "water", "chewing gum").
2. Single ingredients unless they have culinary value (e.g., "garlic").
3. Non-food items or items that do not constitute a recipe (e.g., "salt", "sugar").
4. Raw vegetables, raw fruits, raw meat
5. Condiments
6. Baby foods

Recommendable items are those that can be used as part of a recipe or a full dish.
"""

file_path = '../processed_data/foods_list.csv'
foods_df = pd.read_csv(file_path)

# use the agent to determine if a food item is recommendable with retry
def is_recommendable(food_desc, retries=3):
    prompt = f"""
    You are a food recommendation agent. Your task is to judge whether a food description should be recommended to users or not.
    Follow the criteria below:
    {recommendable_criteria}

    Food Description: "{food_desc}"
    Should this be recommended? Answer "Yes" or "No" with a brief explanation.
    """
    
    for attempt in range(retries):
        try:
            response = agent.generate_reply(
                messages=[{"content": prompt, "role": "user"}]
            )
            answer = response.lower().strip()
            
            if "yes" in answer:
                return True, answer
            elif "no" in answer:
                return False, answer
            else:
                return False, "Unclear response: " + answer

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for '{food_desc}' with error: {e}")
            time.sleep(2 * (attempt + 1))  

    return False, "Error during evaluation after retries"

# Loop through each row in the dataset and use the agent to classify it
recommendable_foods = []
non_recommendable_foods = []

for _, row in foods_df.iterrows():
    food_desc = row['food_desc']
    
    # Determine if the food is recommendable
    is_recommendable_result, explanation = is_recommendable(food_desc)

    if is_recommendable_result:
        recommendable_foods.append({**row.to_dict(), "explanation": explanation})
        print(f"Recommendable: '{food_desc}' - Explanation: {explanation}")
    else:
        non_recommendable_foods.append({**row.to_dict(), "explanation": explanation})
        print(f"Not Recommendable: '{food_desc}' - Explanation: {explanation}")

    # sleep to avoid rate limiting
    time.sleep(2)

recommendable_foods_df = pd.DataFrame(recommendable_foods)
non_recommendable_foods_df = pd.DataFrame(non_recommendable_foods)

recommendable_foods_df.to_csv('../processed_data/recommendable_foods.csv', index=False)
non_recommendable_foods_df.to_csv('../processed_data/non_recommendable_foods.csv', index=False)

[autogen.oai.client: 09-29 00:57:06] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Not Recommendable: 'Hard candy' - Explanation: no, "hard candy" should not be recommended. it falls under the category of generic items and does not represent a complete food item or dish suitable for recommendation to users.
Not Recommendable: 'Soft drink, root beer' - Explanation: no. this food description should not be recommended because "soft drink, root beer" falls under the category of beverages rather than a complete food item or dish suitable for recommendation.
Recommendable: 'Potato chips, baked, flavored' - Explanation: yes, this food description should be recommended. "potato chips, baked, flavored" represents a complete food item that can be enjoyed as a snack or part of a meal. the description includes preparation method (baked) and additional detail (flavored), making it suitable for recommendation to users looking for snac

In [88]:
recommendable_foods_df.to_csv('../processed_data/recommendable_foods_original.csv', index=False)
non_recommendable_foods_df.to_csv('../processed_data/non_recommendable_foods_original.csv', index=False)

In [91]:
import pandas as pd

# Load the recommendable_foods.csv into a DataFrame
recommendable_foods_df = pd.read_csv('../processed_data/recommendable_foods.csv')

# Load the food_tagging.csv into a DataFrame
food_tagging_df = pd.read_csv('../processed_data/food_tagging.csv')

# Perform a left join on 'food_id'
recommendable_foods_tagging_df = pd.merge(
    recommendable_foods_df,
    food_tagging_df,
    how='left',
    on='food_id'
)

# Save the joined DataFrame to a new CSV file
recommendable_foods_tagging_df.to_csv('../processed_data/recommendable_foods_tagging.csv', index=False)

### **Q&A pairs creation:**

##### Step 1: finding top 5,000 pairs of user-food that have the most macro tags in common (we don't care about the high/low level for now)

*Note: here, I initially used the wrong data: I used the whole users list (96k users) from user_tagging but forgot that we actually only need to look at opioid users and recovered users (i.e. excluding non-opioid users)*

This will be fixed in the next step

In [2]:
import pandas as pd

num_qa = 5000

recommendable_foods_tagging = pd.read_csv('../processed_data/recommendable_foods_tagging.csv') 
food_tagging = pd.read_csv('../processed_data/food_tagging.csv')
user_tagging = pd.read_csv('../processed_data/user_tagging.csv')

# List of macro nutrient columns to consider
macro_columns = [
    'macro_carb', 'macro_phosphorus', 'macro_calorie', 'macro_potassium', 'macro_sodium',
    'macro_cholesterol', 'macro_saturated_fat', 'macro_protein', 'macro_sugar', 'macro_fiber',
    'macro_iron', 'macro_folic_acid', 'macro_vitamin_b12', 'macro_calcium', 'micro_vitamin_d',
    'macro_vitamin_c'
]

# Step 0: Filter food_tagging to only include food_ids that exist in recommendable_foods_tagging
food_tagging = food_tagging[food_tagging['food_id'].isin(recommendable_foods_tagging['food_id'])]

# Step 1: Identify Top num_qa Users with Most `macro_` Tags Equal to 1
user_tagging['macro_tag_count'] = user_tagging[macro_columns].sum(axis=1)
top_users = user_tagging.nlargest(num_qa, 'macro_tag_count')

# Step 2: Identify Top num_qa Foods with Most `macro_` Tags Equal to 1 (after filtering food_tagging)
food_tagging['macro_tag_count'] = food_tagging[macro_columns].sum(axis=1)
top_foods = food_tagging.nlargest(num_qa, 'macro_tag_count')

# Step 3: Perform cross join on the top num_qa users and top num_qa foods
user_macro_df = top_users[['SEQN'] + macro_columns]
food_macro_df = top_foods[['food_id'] + macro_columns]

# Cross join users and foods
user_food_cross = user_macro_df.assign(key=1).merge(food_macro_df.assign(key=1), on='key').drop('key', axis=1)

# Multiply corresponding macro columns and sum them up to calculate matching tags
for col in macro_columns:
    user_food_cross[f'{col}_match'] = user_food_cross[f'{col}_x'] * user_food_cross[f'{col}_y']

user_food_cross['matching_nutrients'] = user_food_cross[[f'{col}_match' for col in macro_columns]].sum(axis=1)

# Sort and select top pairs with the most matching macro tags
top_pairs = user_food_cross.sort_values(by='matching_nutrients', ascending=False).head(num_qa)

# Merge with recommendable_foods_tagging to get food descriptions, and filter out rows where 'food_desc' is NaN
top_with_desc = pd.merge(top_pairs, recommendable_foods_tagging[['food_id', 'food_desc']], on='food_id', how='left')
top_with_desc = top_with_desc[top_with_desc['food_desc'].notna()]

selected_columns = top_with_desc[['SEQN', 'food_id']]
selected_columns.to_csv('../processed_data/user_food_pairs.csv', index=False)

        SEQN   food_id                                          food_desc  \
0      34040  91770020                 Dietetic or low calorie hard candy   
1      34040  95201000  Nutritional powder mix (Carnation Instant Brea...   
2      34040  42203200                                     Soy nut butter   
3      34040  95210020   Nutritional powder mix, high protein (Slim Fast)   
4      34040  57332050           Cereal (General Mills Total Raisin Bran)   
...      ...       ...                                                ...   
4995   63287  57212100                Cereal (General Mills Frankenberry)   
4996  114261  21302000                           Beef, neck bones, cooked   
4997  114261  51109110                               Bread, pita, toasted   
4998  114261  25221520  Salami, made from any type of meat, reduced so...   
4999   83019  92309520             Tea, iced, bottled, green, unsweetened   

      matching_nutrients  
0                     13  
1                    

In [7]:
# selected_columns = top_with_desc[['SEQN', 'food_id']]
# selected_columns.to_csv('../processed_data/user_food_pairs.csv', index=False)

In [11]:
import os
import pandas as pd
import time
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": api_key
}

# food recommendation agent 
agent = ConversableAgent(
    name="food_recommendation_agent",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

recommendable_foods_df = pd.read_csv('../processed_data/recommendable_foods_tagging.csv')
user_tagging_df = pd.read_csv('../processed_data/user_tagging.csv')
user_food_pairs_df = pd.read_csv('../processed_data/user_food_pairs.csv')

# Create a helper function to get the nutrition tags with value 1 for user or food
def get_tags_with_value_1(row, prefix):
    return [col for col in row.index if row[col] == 1 and col.startswith(prefix)]

# Define scoring weights
high_weight_categories = [
    "low_calorie", "high_calorie", "low_protein", "high_protein", "low_carb", "high_carb",
    "low_sugar", "high_sugar", "low_fiber", "high_fiber", "low_saturated_fat", "high_saturated_fat",
    "low_cholesterol", "high_cholesterol", "low_sodium", "high_sodium"
]
low_weight_categories = [
    "low_calcium", "high_calcium", "low_phosphorus", "high_phosphorus", "low_potassium", "high_potassium",
    "low_iron", "high_iron", "low_folic_acid", "high_folic_acid", "low_vitamin_c", "high_vitamin_c",
    "low_vitamin_d", "high_vitamin_d", "low_vitamin_b12", "high_vitamin_b12"
]

# Define a question-answer template
qa_template = """
You must produce output following this template:

Question: For User <SEQN>, given the health profile: <all nutrition tags of the user, i.e. columns in user_tagging starting with prefix user_ and have value of 1>, please judge if Food <food_id> <food_desc>, <all nutrition tags of the food, i.e. columns in recommendable_foods_tagging starting with high_ or low_ and have value of 1>, is a healthy option to the user, and why?

Answer: <Yes/No>, because the food is <high/low> in <nutrients> (these nutrients match with the user's health profile), but <high/low> in <nutrients> (these nutrients are opposite to the user's health profile).
"""

# Iterate over user-food pairs to generate question-answer pairs
question_answer_pairs = []

for _, pair in user_food_pairs_df.iterrows():
    seqn = pair['SEQN']
    food_id = pair['food_id']
    
    # Get user nutritional tags
    user_row = user_tagging_df[user_tagging_df['SEQN'] == seqn]
    if user_row.empty:
        continue
    user_tags = get_tags_with_value_1(user_row.iloc[0], "user_")

    # Get food nutritional tags
    food_row = recommendable_foods_df[recommendable_foods_df['food_id'] == food_id]
    if food_row.empty:
        continue
    food_tags = get_tags_with_value_1(food_row.iloc[0], ("low_", "high_"))

    # Calculate health score
    health_score = 0
    for tag in user_tags:
        food_tag = tag.replace("user_", "")
        
        if food_tag in high_weight_categories:
            if food_tag in food_tags:
                health_score += 10  # Match in high-weight category
            else:
                health_score -= 10  # Mismatch in high-weight category
        elif food_tag in low_weight_categories:
            if food_tag in food_tags:
                health_score += 1  # Match in low-weight category
            else:
                health_score -= 1  # Mismatch in low-weight category

    # Determine the suitability based on health score
    is_healthy = health_score >= 0

    # Create question text
    user_tags_str = ", ".join(user_tags)
    food_desc = food_row.iloc[0]['food_desc']
    food_tags_str = ", ".join(food_tags)
    
    question = (
        f"For User {seqn}, given the health profile: {user_tags_str}, "
        f"please judge if Food {food_id} ({food_desc}), with nutritional properties: {food_tags_str}, "
        "is a healthy option for the user, and why?"
    )

    # Prepare the complete prompt by adding the QA template
    complete_prompt = qa_template + f"\n\n{question}"

    # Generate answer from the score
    answer = "Yes" if is_healthy else "No"

    # Generate reply from agent (LLM should generate a coherent explanation)
    try:
        response = agent.generate_reply(
            messages=[{"content": complete_prompt, "role": "user"}]
        )
        
        formatted_answer = f"{answer}, because {response.strip()}"
        
        question_answer_pairs.append({
            "SEQN": seqn,
            "food_id": food_id,
            "food_desc": food_desc,
            "question": question,
            "answer": formatted_answer
        })
        print(f"Generated answer for SEQN {seqn} and food_id {food_id}: {formatted_answer}")

    except Exception as e:
        print(f"Error generating answer for SEQN {seqn} and food_id {food_id}: {e}")
        continue

    # avoid exceeding rate limits
    time.sleep(2)

qa_pairs_df = pd.DataFrame(question_answer_pairs)

[autogen.oai.client: 09-29 10:36:59] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Generated answer for SEQN 34040 and food_id 91770020: Yes, because Question: For User 34040, given the health profile: user_low_calorie, user_high_potassium, user_low_sodium, user_low_cholesterol, user_low_saturated_fat, user_low_protein, user_high_fiber, user_high_iron, user_high_folate_acid, user_high_vitamin_b12, user_high_calcium, user_high_vitamin_d, user_high_vitamin_c, please judge if Food 91770020 (Dietetic or low calorie hard candy), with nutritional properties: high_calorie, low_protein, high_carb, high_sugar, low_fiber, low_saturated_fat, low_cholesterol, low_sodium, low_calcium, low_phosphorus, low_potassium, low_iron, low_folic_acid, low_vitamin_c, low_vitamin_d, low_vitamin_b12, is a healthy option for the user, and why?

Answer: No, because the food is high in sugar (opposite to user_high_fiber and user_low_sugar) and low in

KeyboardInterrupt: 

In [ ]:
# qa_pairs_df.to_csv('../processed_data/recommendable_foods_question_answers.csv', index=False)
# print("Question-answer pairs saved to '../processed_data/recommendable_foods_question_answers.csv'")

**Using the correct data:** filtering for only active & recovered opioid users by referencing users that are in joined_graph

In [13]:
import torch
import pandas as pd


num_qa = 5000

# Load the joined graph
joined_graph = torch.load('../processed_data/joined_graph.pt')

# Extract the user node IDs from the graph
user_ids_in_graph = joined_graph['user'].node_id.tolist()

# Load the user_tagging.csv file
user_tagging = pd.read_csv('../processed_data/user_tagging.csv')

# Filter user_tagging to include only users that are present in the graph
filtered_user_tagging = user_tagging[user_tagging['SEQN'].isin(user_ids_in_graph)]

# Now proceed with your original code using the filtered_user_tagging DataFrame

# List of macro nutrient columns to consider
macro_columns = [
    'macro_carb', 'macro_phosphorus', 'macro_calorie', 'macro_potassium', 'macro_sodium',
    'macro_cholesterol', 'macro_saturated_fat', 'macro_protein', 'macro_sugar', 'macro_fiber',
    'macro_iron', 'macro_folic_acid', 'macro_vitamin_b12', 'macro_calcium', 'micro_vitamin_d',
    'macro_vitamin_c'
]

# Step 0: Filter food_tagging to only include food_ids that exist in recommendable_foods_tagging
recommendable_foods_tagging = pd.read_csv('../processed_data/recommendable_foods_tagging.csv') 
food_tagging = pd.read_csv('../processed_data/food_tagging.csv')
food_tagging = food_tagging[food_tagging['food_id'].isin(recommendable_foods_tagging['food_id'])]

# Step 1: Identify Top num_qa Users with Most `macro_` Tags Equal to 1
filtered_user_tagging['macro_tag_count'] = filtered_user_tagging[macro_columns].sum(axis=1)
top_users = filtered_user_tagging.nlargest(num_qa, 'macro_tag_count')

# Step 2: Identify Top num_qa Foods with Most `macro_` Tags Equal to 1 (after filtering food_tagging)
food_tagging['macro_tag_count'] = food_tagging[macro_columns].sum(axis=1)
top_foods = food_tagging.nlargest(num_qa, 'macro_tag_count')

# Step 3: Perform cross join on the top num_qa users and top num_qa foods
user_macro_df = top_users[['SEQN'] + macro_columns]
food_macro_df = top_foods[['food_id'] + macro_columns]

# Cross join users and foods
user_food_cross = user_macro_df.assign(key=1).merge(food_macro_df.assign(key=1), on='key').drop('key', axis=1)

# Multiply corresponding macro columns and sum them up to calculate matching tags
for col in macro_columns:
    user_food_cross[f'{col}_match'] = user_food_cross[f'{col}_x'] * user_food_cross[f'{col}_y']

user_food_cross['matching_nutrients'] = user_food_cross[[f'{col}_match' for col in macro_columns]].sum(axis=1)

# Sort and select top pairs with the most matching macro tags
top_pairs = user_food_cross.sort_values(by='matching_nutrients', ascending=False).head(num_qa)

# Merge with recommendable_foods_tagging to get food descriptions, and filter out rows where 'food_desc' is NaN
top_with_desc = pd.merge(top_pairs, recommendable_foods_tagging[['food_id', 'food_desc']], on='food_id', how='left')
top_with_desc = top_with_desc[top_with_desc['food_desc'].notna()]

selected_columns = top_with_desc[['SEQN', 'food_id']]
selected_columns.to_csv('../processed_data/user_food_pairs.csv', index=False)

C:\Users\halan\AppData\Local\Temp\ipykernel_33608\127203116.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_user_tagging['macro_tag_count'] = filtered_user_tagging[macro_columns].sum(axis=1)


In [14]:
display(top_with_desc[['SEQN', 'food_id', 'food_desc', 'matching_nutrients']])

,SEQN,food_id,food_desc,matching_nutrients
0,23816,91770020,Dietetic or low calorie hard candy,12
1,123231,14108060,"Cheese, Parmesan, dry grated, fat free",12
2,123231,91770020,Dietetic or low calorie hard candy,12
3,123231,57332050,Cereal (General Mills Total Raisin Bran),12
4,123231,11830210,"Chocolate beverage powder, light, dry mix, not...",12
...,...,...,...,...
4995,123231,54337100,"Crackers, woven wheat, reduced fat",9
4996,95412,92650210,Cornmeal beverage with chocolate milk,9
4997,47343,57305174,Cereal (Malt-O-Meal Colossal Crunch),9
4998,95656,92191400,"Coffee, instant, pre-sweetened with sugar, not...",9


In [17]:
import os
import pandas as pd
import time
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": api_key
}

# food recommendation agent
agent = ConversableAgent(
    name="food_recommendation_agent",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

recommendable_foods_df = pd.read_csv('../processed_data/recommendable_foods_tagging.csv')
user_tagging_df = pd.read_csv('../processed_data/user_tagging.csv')
user_food_pairs_df = pd.read_csv('../processed_data/user_food_pairs.csv')

# Define important and unimportant tags
important_tags = [
    "low_calorie", "high_calorie", "low_protein", "high_protein", "low_carb", "high_carb",
    "low_sugar", "high_sugar", "low_fiber", "high_fiber", "low_saturated_fat", "high_saturated_fat",
    "low_cholesterol", "high_cholesterol", "low_sodium", "high_sodium"
]
unimportant_tags = [
    "low_calcium", "high_calcium", "low_phosphorus", "high_phosphorus", "low_potassium", "high_potassium",
    "low_iron", "high_iron", "low_folic_acid", "high_folic_acid", "low_vitamin_c", "high_vitamin_c",
    "low_vitamin_d", "high_vitamin_d", "low_vitamin_b12", "high_vitamin_b12"
]

# Create a helper function to get the nutrition tags with value 1 for user or food
def get_tags_with_value_1(row, prefix):
    return [col for col in row.index if row[col] == 1 and col.startswith(prefix)]

# Define a question-answer template
qa_template = """
You must produce output following this template:

Question: For User <SEQN>, given the health profile: <all important nutrition tags of the user, i.e. columns in user_tagging starting with prefix user_ and have value of 1 from the list of important tags>, please judge if Food <food_id> <food_desc>, <all important nutrition tags of the food, i.e. columns in recommendable_foods_tagging starting with high_ or low_ and have value of 1 from the list of important tags>, is a healthy option to the user, and why?
Answer: <Yes/No>, because the food is <high/low> in <nutrients> (these nutrients match with the user's health profile), but <high/low> in <nutrients> (these nutrients are opposite to the user's health profile).
"""

# Iterate over user-food pairs to generate question-answer pairs
question_answer_pairs = []

for _, pair in user_food_pairs_df.iterrows():
    seqn = pair['SEQN']
    food_id = pair['food_id']
    
    # Get user nutritional tags
    user_row = user_tagging_df[user_tagging_df['SEQN'] == seqn]
    if user_row.empty:
        continue
    user_tags = get_tags_with_value_1(user_row.iloc[0], "user_")
    # Filter user_tags to include only important tags
    user_tags = [tag for tag in user_tags if tag.replace("user_", "") in important_tags]

    # Get food nutritional tags
    food_row = recommendable_foods_df[recommendable_foods_df['food_id'] == food_id]
    if food_row.empty:
        continue
    food_tags = get_tags_with_value_1(food_row.iloc[0], ("low_", "high_"))
    # Filter food_tags to include only important tags
    food_tags = [tag for tag in food_tags if tag in important_tags]

    # Create question text
    user_tags_str = ", ".join(user_tags)
    food_desc = food_row.iloc[0]['food_desc']
    food_tags_str = ", ".join(food_tags)
    
    question = (
        f"For User {seqn}, given the health profile: {user_tags_str}, "
        f"please judge if Food {food_id} ({food_desc}), with nutritional properties: {food_tags_str}, "
        "is a healthy option for the user, and why?"
    )

    # Prepare the complete prompt by adding the QA template
    complete_prompt = qa_template + f"\n\n{question}"

    # Generate reply from agent
    try:
        response = agent.generate_reply(
            messages=[{"content": complete_prompt, "role": "user"}]
        )
        
        # Extract the question and answer from the response (since it should be formatted as required)
        question_answer_pairs.append({
            "SEQN": seqn,
            "food_id": food_id,
            "food_desc": food_desc,
            "question": question,
            "answer": response.strip()
        })
        print('*' * 100)
        print(f"Generated answer for SEQN {seqn} and food_id {food_id}:\n\n{response}")

    except Exception as e:
        print(f"Error generating answer for SEQN {seqn} and food_id {food_id}: {e}")
        continue

    # avoid exceeding rate limits
    time.sleep(2)

qa_pairs_df = pd.DataFrame(question_answer_pairs)



[autogen.oai.client: 09-29 10:57:31] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
****************************************************************************************************
Generated answer for SEQN 23816 and food_id 91770020:

Question: For User 23816, given the health profile: user_low_calorie, user_low_sodium, user_low_saturated_fat, user_high_protein, user_low_sugar, user_high_fiber, please judge if Food 91770020 (Dietetic or low calorie hard candy), with nutritional properties: high_calorie, low_protein, high_carb, high_sugar, low_fiber, low_saturated_fat, low_cholesterol, low_sodium, is a healthy option for the user, and why?

Answer: No, because the food is high in calorie (opposite to user_low_calorie), low in protein (opposite to user_high_protein), high in sugar (opposite to user_low_sugar), low in fiber (opposite to user_high_fiber), low in saturated fat (matches user_low_saturated_fat), low in chol

KeyboardInterrupt: 

In [ ]:
qa_pairs_df.to_csv('../processed_data/recommendable_foods_question_answers.csv', index=False)